In [1]:
## Created by Wentinn Liao

# CS180 Project 2

In [ ]:
!pip install numpy imageio scikit-learn scikit-image torch pytorch-ignite opencv-python matplotlib mpl_interactions PyQt6

In [ ]:
#@title Mount Google Drive
import os
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [4]:
#@title Symlink Setup
def ptpp(PATH: str) -> str: # Converts path to python path
    return PATH.replace('\\', '')

DRIVE_PATH = '/content/gdrive/My\ Drive/CS180/HW2'
if not os.path.exists(ptpp(DRIVE_PATH)):
    %mkdir $DRIVE_PATH
SYM_PATH = '/content/HW2'
if not os.path.exists(ptpp(SYM_PATH)):
    !ln -s $DRIVE_PATH $SYM_PATH
%cd $SYM_PATH

/content/gdrive/My Drive/CS180/HW2


In [5]:
import sys
sys.path.append(ptpp(SYM_PATH) + '/code')

In [7]:
#@title Configure Jupyter Notebook
%load_ext autoreload
%autoreload 2

In [8]:
#@title Library Setup
import math
import numpy as np
import scipy as sc
import skimage as sk
import skimage.io as skio
import skimage.transform as sktr
import cv2
import random
import torch

seed = 7
torch.manual_seed(seed)
random.seed(seed)
torch.set_default_dtype(torch.double)

# Implementation

In [9]:
#@title Utilities
def read_image(imname: str) -> np.ndarray:
    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)
    im = sk.util.img_as_float(im)
    return im

def im_to_uint8(im: np.ndarray) -> np.ndarray:
    return np.floor(256 * im).astype('u1')

def im_rescale(im):
    lo = np.min(im)
    hi = np.max(im)
    return (im - lo) / (hi - lo)

def im_saturate(im):
    return np.stack([im_rescale(im[:, :, c]) for c in range(im.shape[2])], axis=2)

# Part 1: Fun with Filters

In [ ]:
#@title Part 1.1: Finite Difference Operator

threshold = 0.23

im = read_image('code/data/cameraman.jpg')[:, :, 0]

Dx = np.array([[1.], [-1.]])
Dy = np.array([[1., -1.]])

Dim_Dx = sc.signal.convolve2d(im, Dx, mode='same')
Dim_Dy = sc.signal.convolve2d(im, Dy, mode='same')

print('Finite difference with respect to x')
skio.imshow(Dim_Dx)
skio.show()

print('Finite difference with respect to y')
skio.imshow(Dim_Dy)
skio.show()

print('Gradient magnitude')
edge_im = np.linalg.norm(np.stack([Dim_Dx, Dim_Dy]), axis=0)
skio.imshow(edge_im)
skio.show()

print('Binary gradient magnitude')
binary_edge_im = edge_im > threshold
skio.imshow(binary_edge_im)
skio.show()

In [ ]:
#@title Part 1.2: Derivative of Gaussian (DoG) Filter

threshold = 0.1

im = read_image('code/data/cameraman.png')[:, :, 0]

Dx = np.array([[1.], [-1.]])
Dy = np.array([[1., -1.]])
G = (G := cv2.getGaussianKernel(8, 1)) @ G.T

# Separate Convolutions
blurred_im = sc.signal.convolve2d(im, G, mode='same')

Dblurred_im_Dx = sc.signal.convolve2d(blurred_im, Dx, mode='same')
Dblurred_im_Dy = sc.signal.convolve2d(blurred_im, Dy, mode='same')

print('Edges as gradient magnitude of blurred image')
edge_blurred_im = np.linalg.norm(np.stack([Dblurred_im_Dx, Dblurred_im_Dy]), axis=0)
skio.imshow(edge_blurred_im)
skio.show()

print('Binary separate convolutions')
binary_edge_blurred_im = edge_blurred_im > threshold
skio.imshow(binary_edge_blurred_im)
skio.show()

# Single Convolution
DxG = sc.signal.convolve2d(G, Dx, mode='same')
DyG = sc.signal.convolve2d(G, Dy, mode='same')

Dim_DxG = sc.signal.convolve2d(im, DxG, mode='same')
Dim_DyG = sc.signal.convolve2d(im, DyG, mode='same')

print('Edges as DoG of image')
edge_blurred_im2 = np.linalg.norm(np.stack([Dim_DxG, Dim_DyG]), axis=0)
skio.imshow(edge_blurred_im2)
skio.show()

print('Binary single convolutions')
binary_edge_blurred_im2 = edge_blurred_im2 > threshold
skio.imshow(binary_edge_blurred_im2)
skio.show()

# Part 2: Fun with Frequencies!

In [ ]:
#@title Part 2.1: Image "Sharpening"

sharpening = 1
images = [
    'taj.jpg',
    'genki 1.jpg',
    'hadelich.jpg',
    'benny.jpg'
]

G = (G := cv2.getGaussianKernel(7, 1)) @ G.T
I = (v := np.array([[0., 0., 0., 1., 0., 0., 0.]])).T @ v
S = (I + sharpening * (I - G))[:, :, None]

for image in images:
    print(f'Original {image}')
    im = read_image(f'code/data/{image}')
    skio.imshow(im)
    skio.show()

    print(f'Sharpened {image}')
    sharpened_im = np.clip(sc.signal.convolve(im, S, mode='same'), 0, 1)
    skio.imshow(sharpened_im)
    skio.show()

    skio.imsave(f'images/Sharpened {image}', sk.img_as_ubyte(sharpened_im), quality=100)

In [ ]:
print(f'Original image')
im = read_image(f'code/data/genki 2.jpg')
skio.imshow(im)
skio.show()

print('Blurred and resharpened image')
blurred_im = sc.signal.convolve(im, G[:, :, None], mode='same')
resharpened_im = np.clip(sc.signal.convolve(blurred_im, S, mode='same'), 0, 1)
skio.imshow(resharpened_im)
skio.show()

print((np.linalg.norm(im - resharpened_im) ** 2) / np.prod(im.shape))

## Note: Part 2.2 does not run on Colab because pop-up/interactive windows are disabled

In [ ]:
#@title Part 2.2: Hybrid Images
import hybrid_python.align_image_code as lib

im_pairs = [
    ['code/data/joy.jpg', 'code/data/genki 1.jpg', 18],
    ['code/data/eddy chen.jpg', 'code/data/brett yang.jpg', 9],
    ['code/data/miyu 1.jpg', 'code/data/miyu 2.jpg', 50],
    ['code/data/wengenn.jpg', 'code/data/dad.jpg', 20]
]

for im1name, im2name, sigma in im_pairs:
    %matplotlib osx
    im1, im2 = lib.align_images(read_image(im1name), read_image(im2name))

    %matplotlib inline
    k = max(int(3 * sigma), 3)
    G = (G := cv2.getGaussianKernel(2 * k + 1, sigma)) @ G.T

    v = np.zeros((2 * k + 1, 1), dtype=float)
    v[k] = 1.
    H = v @ v.T - G

    high_frequency_im1 = sc.signal.convolve(im1, H[:, :, None], mode='same')
    low_frequency_im2 = sc.signal.convolve(im2, G[:, :, None], mode='same')

    print(f'High frequency {im1name}')
    skio.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(high_frequency_im1)))))
    skio.show()

    print(f'Low frequency {im2name}')
    skio.imshow(np.log(np.abs(np.fft.fftshift(np.fft.fft2(low_frequency_im2)))))
    skio.show()

    print('Hybrid image')
    hybrid_im = np.clip(high_frequency_im1 + low_frequency_im2, 0, 1)
    skio.imshow(hybrid_im)
    skio.show()

In [51]:
#@title Part 2.3: Gaussian and Laplacian Stacks

def gaussian_stack(im, depth=8, sigma=1):
    gaussian_stack = [im]
    for _ in range(depth):
        k = int(math.ceil(3 * sigma))
        G = ((G := cv2.getGaussianKernel(2 * k + 1, sigma)) @ G.T)[:, :, None]

        gaussian_stack.append(sc.signal.convolve(gaussian_stack[-1], G, mode='same'))
        sigma *= 2
    return gaussian_stack

def gaussian_and_laplacian_stack(im, depth=8, sigma=1):
    gaussian_stack, laplacian_stack = [im], []
    for _ in range(depth + 1):
        k = int(math.ceil(3 * sigma))
        G = ((G := cv2.getGaussianKernel(2 * k + 1, sigma)) @ G.T)[:, :, None]

        gaussian_stack.append(sc.signal.convolve(gaussian_stack[-1], G, mode='same'))
        laplacian_stack.append(gaussian_stack[-2] - gaussian_stack[-1])

        sigma *= 2
    gaussian_stack.pop()
    return gaussian_stack, laplacian_stack

def blend(im1, im2, mask, im1name=None, im2name=None, depth=8, sigma=1):
    mask_gaussian_stack = gaussian_stack(mask, depth=depth)

    im1_gaussian_stack, im1_laplacian_stack = gaussian_and_laplacian_stack(im1, depth=depth)
    im2_gaussian_stack, im2_laplacian_stack = gaussian_and_laplacian_stack(im2, depth=depth)

    blend_stack = []
    for d in range(depth + 1):
        l1, l2 = im1_laplacian_stack[d], im2_laplacian_stack[d]
        mask = mask_gaussian_stack[d]

        masked_l1, masked_l2 = mask * l1, (1 - mask) * l2
        blend_stack.append(blend := masked_l1 + masked_l2)

        im_to_save = im_rescale(np.concatenate([l1, masked_l1, blend, masked_l2, l2], axis=1))
        if im1name is not None and im2name is not None:
            skio.imsave(f'images/{im1name}+{im2name} frequency blend depth {d}.png', sk.img_as_ubyte(im_to_save))
        skio.imshow(im_to_save)
        skio.show()

    blended_im = np.clip(np.sum(blend_stack, axis=0), 0, 1)
    skio.imshow(np.concatenate([im1, blended_im, im2], axis=1))
    skio.show()

    return blended_im

In [ ]:
im1 = read_image('code/spline/apple.jpeg')
im2 = read_image('code/spline/orange.jpeg')

h, w, c = im1.shape
mask = np.concatenate([np.ones((h, int(math.floor(w / 2)), c)), np.zeros((h, int(math.ceil(w / 2)), c))], axis=1)

blended_im = blend(im1, im2, mask, 'apple', 'orange')

In [ ]:
#@title Part 2.4: Multiresolution Blending (a.k.a. the oraple!)

rescale_factor = 0.4
miyu_mask = np.round(sktr.rescale(read_image('code/data/miyu face mask.png')[:, :, :-1], rescale_factor, channel_axis=-1))
dad_snacks = sktr.rescale(read_image('code/data/padded dad snacks.jpg'), rescale_factor, channel_axis=-1)
miyu = sktr.rescale(read_image('code/data/padded miyu face.jpg'), rescale_factor, channel_axis=-1)

miyu_snacks = blend(dad_snacks, miyu, miyu_mask, im1name='Dad Snacks', im2name='Miyu', depth=10, sigma=1.5)
skio.imshow(miyu_snacks)
skio.show()

skio.imsave('images/Miyu Snacks.png', sk.img_as_ubyte(miyu_snacks))

In [ ]:
rescale_factor=0.5

aurora = sktr.rescale(read_image('code/data/aurora.jpg'), rescale_factor, channel_axis=-1)
night = sktr.rescale(read_image('code/data/starry night.jpg')[1:-1], rescale_factor, channel_axis=-1)

h, w, c = aurora.shape
mask = np.concatenate([np.ones((int(math.floor(h / 2)), w, c)), np.zeros((int(math.ceil(h / 2)), w, c))], axis=0)

aurora_night = blend(aurora, night, mask, 'Aurora', 'Starry Night', depth=10, sigma=2)

skio.imshow(aurora_night)
skio.show()

skio.imsave('images/Aurora Night.png', sk.img_as_ubyte(aurora_night))

In [ ]:
rescale_factor = 0.4
joy_mask = np.round(sktr.rescale(read_image('code/data/joy face mask.png')[:, :, :-1], rescale_factor, channel_axis=-1))
school = sktr.rescale(read_image('code/data/school.jpg'), rescale_factor, channel_axis=-1)
joy = sktr.rescale(read_image('code/data/padded joy face.png')[:, :, :-1], rescale_factor, channel_axis=-1)

school_joy = blend(school, joy, joy_mask, im1name='School', im2name='Joy', depth=10, sigma=1.5)
skio.imshow(school_joy)
skio.show()

skio.imsave('images/School Joy.png', sk.img_as_ubyte(school_joy))